In [1]:
# Zelle 1 - Pakete

import math
from dataclasses import dataclass
from typing import List, Dict
import json
from pathlib import Path

import pandas as pd


In [2]:
# ----------------------------------------------------------
# Zelle 2 - Input: Excel einlesen und Prozessdaten erzeugen
# ----------------------------------------------------------


#  Datei-Pfad angeben (bitte anpassen)
path = r"C:/Users/.../Masterarbeit/Python/"
#excel_path = r"C:/Users/Pablo/Documents/Uni/Masterarbeit/Python/Prozesse.xlsx"
input_folger = r"Kostenmodell x10/"
excel_file = r"Prozesslaufzeiten und Konfig x10.xlsx"
excel_path = path + input_folger + excel_file

#  Excel einlesen
#df_prozesse_raw = pd.read_excel(excel_path, sheet_name="Monatslaufzeiten")
df_prozesse_raw = pd.read_excel(excel_path, sheet_name="Jahreslaufzeiten")


df_prozesse_raw = df_prozesse_raw.dropna(subset=["xProzess_ID"])
df_prozesse_raw["xProzess_ID"] = df_prozesse_raw["xProzess_ID"].astype(int)

df_prozesse_raw = df_prozesse_raw.rename(columns={"Laufzeit": "Laufzeit"})
df_prozesse_raw = df_prozesse_raw.rename(columns={"xProzess_ID": "Prozess"})

df_prozesse_raw = df_prozesse_raw[["Prozess", "Month", "Laufzeit"]]

df_prozesse = (df_prozesse_raw.groupby("Prozess", as_index=False).agg(Laufzeit=("Laufzeit", "mean")))
#df_prozesse = df_prozesse_raw[["Prozess", "Laufzeit"]]

display(df_prozesse)

,Prozess,Laufzeit
0,10000,168.057070
1,10001,154.393829
2,10002,102.313037
3,10003,100.970770
4,10004,92.287095
...,...,...
1895,19185,0.006664
1896,19186,0.014815
1897,19187,0.025000
1898,19188,0.007734


In [3]:
# ----------------------------------------------------------
# Zelle 3 - Config
# ----------------------------------------------------------

# Maximale Laufzeit pro VDI / Bot pro Tag (Stunden)
MAX_HOURS_PER_DAY = 20

# Pool-Größen (Anzahl Prozesse, die sich eine VDI teilen dürfen)
POOL_SIZES = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 32, 64, 128, 256, 512, 1000, 5000]

# Exponentielle Basen für Score-Greedy
EXP_BASES = [1.1, 1.2, 1.25, 2.0]

# Output-Pfad für Konstellationen
OUTPUT_FOLDER = Path(path + input_folger + "JSON")

# Steuerung: welche Verfahren sollen ausgeführt werden
ASSIGNMENT_METHODS = [
    "greedy",
    "score_greedy",
    "preseeding_best_fit",
    "pairing_top90",
]


# Optional: 0er-Pool überspringen
SKIP_POOL_SIZE_ZERO = True


In [4]:
df = df_prozesse.copy()

In [5]:
# ----------------------------------------------------------
# Zelle 4 - Zuweisungsverfahren
# ----------------------------------------------------------


def assign_greedy(df: pd.DataFrame, pool_size: int, max_hours_per_day: float) -> dict:
    """
    Greedy-Zuweisung: Prozesse werden nach Laufzeit sortiert
    und sequenziell zu Gruppen zusammengefasst.
    """

    pool_max_hours_per_day = max_hours_per_day * pool_size

    # Sonderfall: pool_size == 0 → alle Prozesse solo
    if pool_size == 0:
        konstellation = [[int(pid)] for pid in df["Prozess"]]
        return {
            "verfahren": "greedy",
            "pool_size": pool_size,
            "titel": f"Greedy_Pool{pool_size}",
            "notiz": "Poolgröße 0: alle Prozesse einzeln.",
            "konstellation": konstellation,
        }

    # Prozesse trennen
    solo_df = df[df["Laufzeit"] > pool_max_hours_per_day].copy()
    pool_df = df[df["Laufzeit"] <= pool_max_hours_per_day].copy()

    # Sortierung für bessere Packung
    pool_df = pool_df.sort_values("Laufzeit", ascending=False)

    groups = []
    current_group = []
    current_hours = 0.0

    for _, row in pool_df.iterrows():
        runtime = row["Laufzeit"]
        process = int(row["Prozess"])

        if current_hours + runtime <= pool_max_hours_per_day:
            current_group.append(process)
            current_hours += runtime
        else:
            groups.append(current_group)
            current_group = [process]
            current_hours = runtime

    if current_group:
        groups.append(current_group)

    konstellation = []
    konstellation.extend(groups)

    for _, row in solo_df.iterrows():
        konstellation.append([int(row["Prozess"])])

    return {
        "verfahren": "greedy",
        "pool_size": pool_size,
        "titel": f"Greedy_Poolsize{pool_size}",
        "notiz": f"Greedy-Zuweisung mit Poolgröße {pool_size}.",
        "konstellation": konstellation,
    }


def _assign_score_greedy_core(
    df: pd.DataFrame,
    pool_size: int,
    max_hours_per_day: float,
    exp_base: float,
) -> dict:
    """
    Core-Funktion: Score-Greedy mit fester exponentieller Basis.
    """

    pool_max_hours_per_day = max_hours_per_day * pool_size

    if pool_size == 0:
        konstellation = [[int(pid)] for pid in df["Prozess"]]
        return {
            "verfahren": "score_greedy",
            "pool_size": pool_size,
            "exp_base": exp_base,
            "titel": f"ScoreGreedy_b{exp_base}_Poolsize{pool_size}",
            "notiz": "Poolgröße 0: alle Prozesse einzeln.",
            "konstellation": konstellation,
        }

    solo_df = df[df["Laufzeit"] > pool_max_hours_per_day].copy()
    pool_df = df[df["Laufzeit"] <= pool_max_hours_per_day].copy()

    groups = []

    # 🔹 Solo-Prozesse initialisieren
    for _, row in solo_df.iterrows():
        groups.append({
            "prozesse": [int(row["Prozess"])],
            "hours": float(row["Laufzeit"]),
        })

    pool_df = pool_df.sort_values("Laufzeit", ascending=False)

    # 🔹 Score-Greedy-Zuweisung
    for _, row in pool_df.iterrows():
        runtime = float(row["Laufzeit"])
        process = int(row["Prozess"])

        best_group = None
        best_score = None

        for group in groups:
            if group["hours"] + runtime > pool_max_hours_per_day:
                continue

            n = len(group["prozesse"])

            try:
                score = exp_base ** n
            except OverflowError:
                # Diese zu großen Gruppierungsoption ignorieren
                continue

            if best_score is None or score < best_score:
                best_group = group
                best_score = score

        # 🔹 Zuweisung
        if best_group is not None:
            best_group["prozesse"].append(process)
            best_group["hours"] += runtime
        else:
            groups.append({
                "prozesse": [process],
                "hours": runtime,
            })

    konstellation = [g["prozesse"] for g in groups]

    return {
        "verfahren": "score_greedy",
        "pool_size": pool_size,
        "exp_base": exp_base,
        "titel": f"ScoreGreedy_b{exp_base}_Poolsize{pool_size}",
        "notiz": (
            f"Score-Greedy mit exponentiellen Grenzkosten "
            f"(Basis={exp_base}, Poolgröße={pool_size})."
        ),
        "konstellation": konstellation,
    }


def assign_score_greedy(
    df: pd.DataFrame,
    pool_size: int,
    max_hours_per_day: float,
    exp_bases: list[float],
) -> list[dict]:
    """
    Wrapper: führt Score-Greedy für mehrere exponentielle Basen aus.
    """
    results = []

    for exp_base in exp_bases:
        result = _assign_score_greedy_core(
            df=df,
            pool_size=pool_size,
            max_hours_per_day=max_hours_per_day,
            exp_base=exp_base,
        )
        results.append(result)

    return results


def assign_preseeding_best_fit(
    df: pd.DataFrame,
    pool_size: int,
    max_hours_per_day: float,
) -> dict:
    """
    Pre-Seeding Best-Fit:
    - Prozesse, die nicht pool-fähig sind, starten eigene Pools
    - restliche Prozesse werden per Best-Fit (min. Restkapazität) zugewiesen
    """

    pool_max_hours_per_day = max_hours_per_day * pool_size

    # Sonderfall: pool_size == 0 → alles solo
    if pool_size == 0:
        konstellation = [[int(pid)] for pid in df["Prozess"]]
        return {
            "verfahren": "preseeding_best_fit",
            "pool_size": pool_size,
            "titel": f"PreSeedingBestFit_Poolsize{pool_size}",
            "notiz": "Poolgröße 0: alle Prozesse einzeln.",
            "konstellation": konstellation,
        }

    # Prozesse trennen
    solo_df = df[df["Laufzeit"] > pool_max_hours_per_day].copy()
    pool_df = df[df["Laufzeit"] <= pool_max_hours_per_day].copy()

    pools = []

    # 🔹 Pre-Seeding mit Solo-Prozessen
    for _, row in solo_df.iterrows():
        pools.append({
            "prozesse": [int(row["Prozess"])],
            "hours": float(row["Laufzeit"]),
        })

    # 🔹 Lange Prozesse zuerst (klassisch für Best-Fit)
    pool_df = pool_df.sort_values("Laufzeit", ascending=False)

    # 🔹 Best-Fit-Zuweisung
    for _, row in pool_df.iterrows():
        runtime = float(row["Laufzeit"])
        process = int(row["Prozess"])

        best_pool = None
        best_remaining_capacity = None

        for pool in pools:
            remaining_capacity = pool_max_hours_per_day - pool["hours"]

            if remaining_capacity >= runtime:
                if (
                    best_remaining_capacity is None
                    or remaining_capacity < best_remaining_capacity
                ):
                    best_pool = pool
                    best_remaining_capacity = remaining_capacity

        if best_pool is not None:
            best_pool["prozesse"].append(process)
            best_pool["hours"] += runtime
        else:
            pools.append({
                "prozesse": [process],
                "hours": runtime,
            })

    konstellation = [pool["prozesse"] for pool in pools]

    return {
        "verfahren": "preseeding_best_fit",
        "pool_size": pool_size,
        "titel": f"PreSeedingBestFit_Poolsize{pool_size}",
        "notiz": (
            "Best-Fit-Verfahren mit Pre-Seeding: "
            "nicht pool-fähige Prozesse starten eigene Pools."
        ),
        "konstellation": konstellation,
    }


def assign_pairing_top90(
    df: pd.DataFrame,
    pool_size: int,
    max_hours_per_day: float,
    top_exclude_ratio: float = 0.10,
) -> dict:
    """
    Pairing-Verfahren:
    - Top X% der Prozesse (nach Laufzeit) werden ausgeschlossen (solo)
    - Restliche Prozesse werden paarweise gruppiert
    - Maximal `pool_size` Paare
    """

    # Sonderfall
    if pool_size == 0:
        konstellation = [[int(pid)] for pid in df["Prozess"]]
        return {
            "verfahren": "pairing_top90",
            "pool_size": pool_size,
            "titel": f"PairingTop90_Poolsize{pool_size}",
            "notiz": "Poolgröße 0: alle Prozesse einzeln.",
            "konstellation": konstellation,
        }

    # 🔹 Sortieren nach Laufzeit (absteigend)
    df_sorted = df.sort_values("Laufzeit", ascending=False).reset_index(drop=True)

    n_total = len(df_sorted)
    n_exclude = int(math.ceil(n_total * top_exclude_ratio))

    # 🔹 Top 10 % → solo
    excluded_df = df_sorted.iloc[:n_exclude]
    candidate_df = df_sorted.iloc[n_exclude:]

    konstellation = []

    # Solo-Gruppen (Top 10 %)
    for _, row in excluded_df.iterrows():
        konstellation.append([int(row["Prozess"])])

    # 🔹 Pairing-Kandidaten sortieren (aufsteigend für Two-Pointer)
    candidates = candidate_df.sort_values("Laufzeit").reset_index(drop=True)

    left = 0
    right = len(candidates) - 1
    pairs_formed = 0

    used = set()

    while left < right and pairs_formed < pool_size:
        p_left = candidates.iloc[left]
        p_right = candidates.iloc[right]

        runtime_sum = p_left["Laufzeit"] + p_right["Laufzeit"]

        if runtime_sum <= max_hours_per_day:
            konstellation.append([
                int(p_left["Prozess"]),
                int(p_right["Prozess"]),
            ])
            pairs_formed += 1
            left += 1
            right -= 1
        else:
            # Längster Prozess passt mit niemandem → solo
            konstellation.append([int(p_right["Prozess"])])
            right -= 1

    # 🔹 Übrige Kandidaten → solo
    for i in range(left, right + 1):
        konstellation.append([int(candidates.iloc[i]["Prozess"])])

    return {
        "verfahren": "pairing_top90",
        "pool_size": pool_size,
        "titel": f"PairingTop90_Poolsize{pool_size}",
        "notiz": (
            f"Paarungsbasiertes Verfahren: Top {int(top_exclude_ratio*100)}% "
            "der Prozesse solo, Rest paarweise gruppiert "
            f"(max. {pool_size} Paare)."
        ),
        "konstellation": konstellation,
    }



ASSIGNMENT_FUNCTIONS = {
    "greedy": assign_greedy,
    "score_greedy": assign_score_greedy,
    "preseeding_best_fit": assign_preseeding_best_fit,
    "pairing_top90": assign_pairing_top90,
}




In [6]:
# ----------------------------------------------------------
# Zelle 5 - Orchestrierung & Export
# ----------------------------------------------------------

OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

export_index = []  # Überblick über alle erzeugten Konstellationen

for pool_size in POOL_SIZES:

    if SKIP_POOL_SIZE_ZERO and pool_size == 0:
        continue

    for method_name in ASSIGNMENT_METHODS:

        assign_fn = ASSIGNMENT_FUNCTIONS[method_name]

        # Verfahren ausführen
        result = assign_fn(
            df=df_prozesse,
            pool_size=pool_size,
            max_hours_per_day=MAX_HOURS_PER_DAY,
            **(
                {"exp_bases": EXP_BASES}
                if method_name == "score_greedy"
                else {}
            )
        )

        # Einheitliche Behandlung (einzelnes Dict oder Liste von Dicts)
        results = result if isinstance(result, list) else [result]

        for r in results:

            exp_base = r.get("exp_base")  # None bei Greedy

            # JSON-Dateiname (inkl. Basis, falls vorhanden)
            json_filename = (
                f"konstellation_{r['verfahren']}"
                f"_poolsize{pool_size}"
                + (f"_b{exp_base}" if exp_base is not None else "")
                + ".json"
            )

            json_path = OUTPUT_FOLDER / json_filename

            # JSON schreiben
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(r, f, indent=2, ensure_ascii=False)

            print(f"✔ Exportiert: {json_filename}")

            # Index-Eintrag
            export_index.append({
                "verfahren": r["verfahren"],
                "pool_size": pool_size,
                "exp_base": exp_base,
                "titel": r["titel"],
                "datei": json_filename,
                "anzahl_gruppen": len(r["konstellation"]),
            })

print(json_path)

✔ Exportiert: konstellation_greedy_poolsize1.json
✔ Exportiert: konstellation_score_greedy_poolsize1_b1.1.json
✔ Exportiert: konstellation_score_greedy_poolsize1_b1.2.json
✔ Exportiert: konstellation_score_greedy_poolsize1_b1.25.json
✔ Exportiert: konstellation_score_greedy_poolsize1_b2.0.json
✔ Exportiert: konstellation_preseeding_best_fit_poolsize1.json
✔ Exportiert: konstellation_pairing_top90_poolsize1.json
✔ Exportiert: konstellation_greedy_poolsize2.json
✔ Exportiert: konstellation_score_greedy_poolsize2_b1.1.json
✔ Exportiert: konstellation_score_greedy_poolsize2_b1.2.json
✔ Exportiert: konstellation_score_greedy_poolsize2_b1.25.json
✔ Exportiert: konstellation_score_greedy_poolsize2_b2.0.json
✔ Exportiert: konstellation_preseeding_best_fit_poolsize2.json
✔ Exportiert: konstellation_pairing_top90_poolsize2.json
✔ Exportiert: konstellation_greedy_poolsize4.json
✔ Exportiert: konstellation_score_greedy_poolsize4_b1.1.json
✔ Exportiert: konstellation_score_greedy_poolsize4_b1.2.jso

In [7]:
# ----------------------------------------------------------
# Zelle 6 - Zusammenfassung: Pool-reichstes & Pool-ärmstes Ergebnis
# ----------------------------------------------------------

if export_index:

    # Pool-reichstes Ergebnis (meiste Gruppen)
    max_pools_result = max(export_index, key=lambda x: x["anzahl_gruppen"])

    # Pool-ärmstes Ergebnis (wenigste Gruppen)
    min_pools_result = min(export_index, key=lambda x: x["anzahl_gruppen"])

    print("\n📊 Zusammenfassung der Extremwerte:\n")
    print(f"  Anzahl aller Konstellationen : {len(export_index)}\n")

    print("🔺 Pool-reichstes Ergebnis:")
    print(f"  Verfahren        : {max_pools_result['verfahren']}")
    print(f"  Pool-Größe       : {max_pools_result['pool_size']}")
    print(f"  Exponentielle Basis : {max_pools_result.get('exp_base')}")
    print(f"  Anzahl Pools     : {max_pools_result['anzahl_gruppen']} (vgl. Anzahl Prozesse: {len(df)})")
    print(f"  Datei            : {max_pools_result['datei']}\n")

    print("🔻 Pool-ärmstes Ergebnis:")
    print(f"  Verfahren        : {min_pools_result['verfahren']}")
    print(f"  Pool-Größe       : {min_pools_result['pool_size']}")
    print(f"  Exponentielle Basis : {min_pools_result.get('exp_base')}")
    print(f"  Anzahl Pools     : {min_pools_result['anzahl_gruppen']}")
    print(f"  Datei            : {min_pools_result['datei']}")




📊 Zusammenfassung der Extremwerte:

  Anzahl aller Konstellationen : 140

🔺 Pool-reichstes Ergebnis:
  Verfahren        : pairing_top90
  Pool-Größe       : 1
  Exponentielle Basis : None
  Anzahl Pools     : 1899 (vgl. Anzahl Prozesse: 1900)
  Datei            : konstellation_pairing_top90_poolsize1.json

🔻 Pool-ärmstes Ergebnis:
  Verfahren        : greedy
  Pool-Größe       : 5000
  Exponentielle Basis : None
  Anzahl Pools     : 1
  Datei            : konstellation_greedy_poolsize5000.json
